In [1]:
from sentence_transformers import SentenceTransformer, util

/Users/zoekatz/Desktop/TeamExpenseReportsV2/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import receipt_wrappers as rw

In [48]:
users = pd.read_csv("data/Users.csv")
receipts = rw.Receipt.initialize_batch_receipts("data/",users)
temp = [r.ocr_str for r in receipts]
ocrs = []

for t in temp:
        if t == None:
                ocrs.append("")
        else:
                ocrs.append(t[:120])
        
ocrs_id = [r.doc_id for r in receipts]
usrs = users["vendor_address"] + " " + users["vendor_name"]
usrs_id = users["documentid"].values

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [49]:
ocr_embeddings = model.encode(ocrs, convert_to_tensor=True)


In [12]:
usrs_embeddings = model.encode(usrs, convert_to_tensor=True)

In [50]:
cosine_scores = util.cos_sim(ocr_embeddings, usrs_embeddings)

In [ ]:
len(cosine_scores)

In [51]:
#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, 498):
        cosine_row = cosine_scores[i]
        cosine = cosine_row[j]
        indices = [i, j]
        pairs.append({'index': indices, 'score': cosine})

In [ ]:
#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=True)

for pair in pairs[0:3]:
    i, j = pair['index']
    print("{} || {} || Score: {:.4f}".format(ocrs[i], usrs[j], pair['score']))
    print("\n")

In [53]:
final_pairs = []
matched_usr = []
matched_ocr = []

for pair in pairs:
        i,j = pair['index']
        ocr = ocrs_id[i]
        usr = usrs_id[j]
        if (usr in matched_usr):
                continue
        if (ocr in matched_ocr):
                continue
        matched_usr.append(usr)
        matched_ocr.append(ocr)
        final_pairs.append((ocr, usr))
        if (len(final_pairs) > 498):
                break

In [54]:
len(final_pairs)

376

In [55]:
correct = 0
for x,y in final_pairs:
        if (x == y):
                correct += 1.0
print(correct/376)

0.14893617021276595
